In [30]:
import pandas as pd
import numpy as np
import os
import regex as re
import spacy as sy
import string
from urllib.parse import urlparse
from nltk.tokenize import TweetTokenizer
from deep_translator import GoogleTranslator


nlp_en = sy.load('en_core_web_sm')
all_stopwords = nlp_en.Defaults.stop_words


In [31]:
test_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Sem_Eval/semeval2023task3/preprocessed_data/test.csv')

In [32]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [33]:
test_df.drop(['id', 'keyword', 'location'], axis=1, inplace=True)

In [34]:
test_df.shape

(3263, 1)

In [35]:
def preprocessing(y, df):
    
    pos_tags_final_text = list()
    er_final_text = list()
    preprocessed_text = list()

    for x in df.loc[:,y]:

        tokenizer = TweetTokenizer()
        #tokenizing
        doc = tokenizer.tokenize(x)
        
        # removing links
        tokens = [token for token in doc if not urlparse(token).scheme]
        x = ' '.join(tokens)
        doc = nlp_en(x)
        
        # removing punctuation and white space
        tokens = [token.orth_ for token in doc if not token.is_punct | token.is_space]    
        x = ' '.join(tokens)
        
        # lower case
        x = x.lower()
        doc = nlp_en(x)

        # lemmatization
        tokens = [word.lemma_ for word in doc]   
        x = ' '.join(tokens)
        doc = nlp_en(x)  
        
        # removing punctuation and white space
        tokens = [token.orth_ for token in doc if not token.is_punct | token.is_space]    
        x = ' '.join(tokens)
        doc = nlp_en(x)
        
        # removing individual letters
        tokens = [word.text for word in doc if len(word)>=2]
        x = ' '.join(tokens)  
        
        # removing numbers
        x = re.sub(r'\d+', '', x)
        x = re.sub(' +', ' ', x)
        doc = nlp_en(x)
        
         # removing hashtags and mentions
        tokens = [word.text for word in doc if word.text[0] not in ('#', '@')]
        x = ' '.join(tokens)
        doc = nlp_en(x)
        
        # removing stop words
        tokens = [word for word in doc if not word in all_stopwords]
        list_of_strings  = [i.text for i in tokens]
        x = ' '.join(list_of_strings)
        doc = nlp_en(x)
        
        # Part of speech tagging
        pos_tags = [(i, i.tag_) for i in doc]
        pos_tags_final_text.append(pos_tags)
        
        # entity recognition tagging
        er =  [(i, i.label_, i.label) for i in doc.ents] 
        er_final_text.append(er)
        
        preprocessed_text.append(x)
        
    return pos_tags_final_text, er_final_text, preprocessed_text

In [36]:
pos_tags_final_text, er_final_text, preprocessed_text = preprocessing('text', test_df)

test_df['preprocessed_text'] = preprocessed_text
test_df['pos_tags_text'] = pos_tags_final_text
test_df['er_tags_text'] = er_final_text

In [37]:
filename = '/Users/nitanshjain/Documents/Projects/Sem_Eval/semeval2023task3/preprocessed_data/extra_test.csv'
test_df.to_csv(filename, index=True)